### Quickstart - Dicas básicas para entendimento e início de desenvolvimento

**Importação das bibliotecas que serão utilizadas**

In [1]:
import epmwebapi as epm # biblioteca para interação com o EPM
import datetime
import pytz
import getpass
import numpy as np

**Conectando ao EPM Web Server**

In [2]:
# Método para não expor usuário e senha do EPM no código fonte
user = input('EPM user: ')
password = getpass.getpass("EPM password: ")

# Crie o objeto de conexão informando os endereços do EPM Web Server (Authentication Port e Web API Port), usuário e senha.
connection = epm.EpmConnection('https://localhost:44340', 'https://localhost:44339', user, password)

EPM user:  sa
EPM password:  ········


**Definindo a BasicVariable utilizada nos exemplos a seguir**

In [3]:
# Substitua o nome a variável pela sua
import warnings
 
# Ignorar todos os warnings
warnings.filterwarnings("ignore")
bvname = 'MemoriaEpmServer'
bv = connection.getDataObjects([bvname])

**Definindo o período de consulta a ser utilizado nos exemplos a seguir (com fuso-horário)**

In [4]:
# Definindo fuso-horário a ser utilizado
tz_sp = pytz.timezone('America/Sao_Paulo')

# Definindo intervalo de consulta com base no fuso-horário
end_date = datetime.datetime.now(tz_sp)
ini_date= end_date - datetime.timedelta(hours=1)
queryPeriod = epm.QueryPeriod(ini_date,end_date)

**Leitura de dados históricos brutos (Raw) a BasicVariable informada**

In [5]:
data = bv[bvname].historyReadRaw(queryPeriod)

print(data)

[(6.00000e+01, datetime.datetime(2025, 10, 8, 18, 26, 45, 428000, tzinfo=datetime.timezone.utc), 0)
 (5.00000e+01, datetime.datetime(2025, 10, 8, 19, 14, 2, 583000, tzinfo=datetime.timezone.utc), 0)
 (1.31152e+05, datetime.datetime(2025, 10, 8, 19, 15, 49, 844000, tzinfo=datetime.timezone.utc), 0)
 (1.31712e+05, datetime.datetime(2025, 10, 8, 19, 16, 9, 956000, tzinfo=datetime.timezone.utc), 0)
 (1.30024e+05, datetime.datetime(2025, 10, 8, 19, 16, 19, 995000, tzinfo=datetime.timezone.utc), 0)
 (1.30140e+05, datetime.datetime(2025, 10, 8, 19, 17, 50, 329000, tzinfo=datetime.timezone.utc), 0)
 (1.24480e+05, datetime.datetime(2025, 10, 8, 19, 18, 0, 372000, tzinfo=datetime.timezone.utc), 0)
 (1.24744e+05, datetime.datetime(2025, 10, 8, 19, 18, 50, 570000, tzinfo=datetime.timezone.utc), 0)
 (1.23360e+05, datetime.datetime(2025, 10, 8, 19, 19, 0, 611000, tzinfo=datetime.timezone.utc), 0)
 (1.24200e+05, datetime.datetime(2025, 10, 8, 19, 21, 21, 142000, tzinfo=datetime.timezone.utc), 0)
 (1.

**Leitura usando agregação de dados (Baseados na norma OPC-UA) da BasicVariable informada**  
Para informações sobre os tipos de agregações, consulte o Apêndice A do Guia do Usuário: https://github.com/elipsesoftware/epmprocessor/blob/master/guiadousuario/Agregacoes.md

In [6]:
# Definindo o intervalo de processamento/amostragem (sample interval) 
processInterval = datetime.timedelta(minutes=10)
aggDetails = epm.AggregateDetails(processInterval, epm.AggregateType.Interpolative)

data = bv[bvname].historyReadAggregate(aggDetails, queryPeriod)

print(data)

[(20249.69    , datetime.datetime(2025, 10, 8, 18, 23, 4, 263467, tzinfo=datetime.timezone.utc), 1026)
 (   58.664734, datetime.datetime(2025, 10, 8, 18, 33, 4, 263467, tzinfo=datetime.timezone.utc), 1026)
 (   56.54994 , datetime.datetime(2025, 10, 8, 18, 43, 4, 263467, tzinfo=datetime.timezone.utc), 1026)
 (   54.435146, datetime.datetime(2025, 10, 8, 18, 53, 4, 263467, tzinfo=datetime.timezone.utc), 1026)
 (   52.32035 , datetime.datetime(2025, 10, 8, 19, 3, 4, 263467, tzinfo=datetime.timezone.utc), 1026)
 (   50.205555, datetime.datetime(2025, 10, 8, 19, 13, 4, 263467, tzinfo=datetime.timezone.utc), 1026)]


**Escrita de dados históricos na BasicVariable informada**

In [7]:
# Valores de teste para serem escritos
newvalues = [50,60,30,40,10]

timebase = datetime.datetime.now(tz_sp)
newdates = np.array([timebase + datetime.timedelta(hours=i) for i in range(5)])

# Criando objeto ndarray no formato suportado pelo EPM.
desc = np.dtype([('Value', '>f8'), ('Timestamp', 'object'), ('Quality', 'object')])
datatemp = np.empty(len(newvalues), dtype=desc)

# Looping para preencher os dados.
i=0
while i < len(newvalues):
    datatemp['Value'][i] = newvalues[i]
    datatemp['Timestamp'][i] = newdates[i]
    datatemp['Quality'][i] = 0
    i = i+1

# Gravação de dados no EPM.
bv[bvname].historyUpdate(datatemp)

**Escrita de dados de tempo real em uma BasicVariable**

In [8]:
date = datetime.datetime.now(tz_sp)
value = 100
quality = 0 # O valor 0 (zero) equivale à qualidade boa (Good) em OPC-UA.

bv[bvname].write(value, date, quality)

**Leitura do valor de tempo real da variável**

In [9]:
data = bv[bvname].read()

print(data.value)
print(data.timestamp)

100
2025-10-08 19:23:15.807354+00:00


**Escrevendo Anotações**

A estrutura de dados do EPM permite que se escrevam anotações nas variáveis. A anotação consiste em mensagem, estampa de tempo e usuário. Nesse exemplo vamos escrever uma anotação no valor máximo da variável no período consultado.

In [10]:
data = bv[bvname].historyReadRaw(queryPeriod)

max_value = data['Value'].max()

max_index = np.argwhere(data['Value'] == max_value)

print(max_index)

print(data['Timestamp'][max_index][0])

# Escrevendo anotação
message = 'valor maximo encontrado no periodo'
timestamp = data['Timestamp'][max_index][0][0]
bv[bvname].writeAnnotation(timestamp, message)

[[3]]
[datetime.datetime(2025, 10, 8, 19, 16, 9, 956000, tzinfo=datetime.timezone.utc)]


**Consultando anotações**

In [11]:
annotations = bv[bvname].readAnnotations(ini_date,end_date)
print(annotations)

[(datetime.datetime(2025, 10, 8, 15, 26, 45, 428000, tzinfo=tzoffset(None, -10800)), 'sa', 'valor maximo encontrado no periodo'), (datetime.datetime(2025, 10, 8, 16, 16, 9, 956000, tzinfo=tzoffset(None, -10800)), 'sa', 'valor maximo encontrado no periodo')]


**Fechando a conexão**

É importante fechar a conexão para liberá-la para outros possíveis clients.

In [12]:
connection.close()